# Uniform Kochanek-Bartels Splines (TCB Splines)

Kochanek-Bartels splines are a superset of Cardinal splines which themselves are a superset of [Catmull-Rom splines](catmull-rom-uniform.ipynb).
They have three parameters per vertex (of course they can also be chosen to be the same values for the whole spline).

The parameters are called
$T$ for "tension",
$C$ for "continuity" and
$B$ for "bias".
With the default values of $C = 0$ and $B = 0$, a Kochanek-Bartels spline is identical with a cardinal spline.
If the "tension" parameter also has its default value $T = 0$ it is identical with a Catmull-Rom spline.

Starting point: tangent vector from Catmull-Rom splines:

\begin{equation}
\boldsymbol{\dot{x}}_0 = \frac{
(\boldsymbol{x}_0 - \boldsymbol{x}_{-1}) +
(\boldsymbol{x}_1 - \boldsymbol{x}_0)
}{2}
\end{equation}

## Parameters

### Tension

\begin{equation}
\boldsymbol{\dot{x}}_0 = (1 - T_0) \frac{
(\boldsymbol{x}_0 - \boldsymbol{x}_{-1}) +
(\boldsymbol{x}_1 - \boldsymbol{x}_0)
}{2}
\end{equation}

TODO: comparison with "tension" parameter of cardinal splines

TODO: images

### Continuity

Up to now, the goal was having a continuous second derivative at the control points, i.e. the incoming and outgoing tangent vectors are identical:

\begin{equation}
\boldsymbol{\dot{x}}_0 = \boldsymbol{\dot{x}}_0^{(-)} = \boldsymbol{\dot{x}}_0^{(+)}
\end{equation}

The "continuity" parameter allows us to break this continuity if we so desire:

\begin{align}
\boldsymbol{\dot{x}}_0^{(-)} &= \frac{
(1 - C_0) (\boldsymbol{x}_0 - \boldsymbol{x}_{-1}) +
(1 + C_0) (\boldsymbol{x}_1 - \boldsymbol{x}_0)
}{2}\\
\boldsymbol{\dot{x}}_0^{(+)} &= \frac{
(1 + C_0) (\boldsymbol{x}_0 - \boldsymbol{x}_{-1}) +
(1 - C_0) (\boldsymbol{x}_1 - \boldsymbol{x}_0)
}{2}
\end{align}

When $C_0 = 0$, we are back at a Catmull-Rom spline.
When $C_0 = -1$, we get a tangent like in a piecewise linear curve.
When $C_0 = 1$, we get some weird "inverse corners".

TODO: Example: compare $T_0 = 1$ and $C_0 = -1$: similar shape (a.k.a. "image"), different timing

### Bias

\begin{equation}
\boldsymbol{\dot{x}}_0 = \frac{
(1 + B_0) (\boldsymbol{x}_0 - \boldsymbol{x}_{-1}) +
(1 - B_0) (\boldsymbol{x}_1 - \boldsymbol{x}_0)
}{2}
\end{equation}

### All Three Combined

\begin{align}
\boldsymbol{\dot{x}}_0^{(+)} &= \frac{
(1 - T_0) (1 + C_0) (1 + B_0) (\boldsymbol{x}_0 - \boldsymbol{x}_{-1}) +
(1 - T_0) (1 - C_0) (1 - B_0) (\boldsymbol{x}_1 - \boldsymbol{x}_0)
}{2}\\
\boldsymbol{\dot{x}}_1^{(-)} &= \frac{
(1 - T_1) (1 - C_1) (1 + B_1) (\boldsymbol{x}_1 - \boldsymbol{x}_0) +
(1 - T_1) (1 + C_1) (1 - B_1) (\boldsymbol{x}_2 - \boldsymbol{x}_1)
}{2}
\end{align}

TODO: cite Kochanek and Bartels, equation 9

TODO: cite Kochanek and Bartels, equation 8

Note: There is an error in eq (6.11) of Ian Millington's paper (all subscripts of $x$ are wrong, most likely copy-pasted from the preceding equation).

To simplify the result we will get later, we introduce the following shorthands (as suggested in Millington's paper):

\begin{align}
a &= (1 - T_0) (1 + C_0) (1 + B_0)\\
b &= (1 - T_0) (1 - C_0) (1 - B_0)\\
c &= (1 - T_1) (1 - C_1) (1 + B_1)\\
d &= (1 - T_1) (1 + C_1) (1 - B_1)
\end{align}

This leads to the simplified equations

\begin{align}
\boldsymbol{\dot{x}}_0^{(+)} &= \frac{
a (\boldsymbol{x}_0 - \boldsymbol{x}_{-1}) +
b (\boldsymbol{x}_1 - \boldsymbol{x}_0)
}{2}\\
\boldsymbol{\dot{x}}_1^{(-)} &= \frac{
c (\boldsymbol{x}_1 - \boldsymbol{x}_0) +
d (\boldsymbol{x}_2 - \boldsymbol{x}_1)
}{2}
\end{align}

## Calculation

In [ ]:
%matplotlib inline
from IPython.display import display
import sympy as sp
sp.init_printing()

In [ ]:
from utility import NamedExpression, NamedMatrix

Same control values as Catmull-Rom ...

In [ ]:
x_1, x0, x1, x2 = sp.symbols('xbm_-1 xbm:3')

In [ ]:
control_values_KB = sp.Matrix([x_1, x0, x1, x2])
control_values_KB

... but three additional parameters per vertex.
In our calculation, the parameters belonging to $\boldsymbol{x}_0$ and $\boldsymbol{x}_1$ are relevant:

In [ ]:
T0, T1 = sp.symbols('T:2')
C0, C1 = sp.symbols('C:2')
B0, B1 = sp.symbols('B:2')

In [ ]:
a = NamedExpression('a', (1 - T0) * (1 + C0) * (1 + B0))
b = NamedExpression('b', (1 - T0) * (1 - C0) * (1 - B0))
c = NamedExpression('c', (1 - T1) * (1 - C1) * (1 + B1))
d = NamedExpression('d', (1 - T1) * (1 + C1) * (1 - B1))
display(a, b, c, d)

In [ ]:
xd0 = NamedExpression('xdotbm0', sp.S.Half * (a.name * (x0 - x_1) + b.name * (x1 - x0)))
xd1 = NamedExpression('xdotbm1', sp.S.Half * (c.name * (x1 - x0)  + d.name * (x2 - x1)))
display(xd0, xd1)

In [ ]:
display(xd0.subs([a, b]))
display(xd1.subs([c, d]))

Same as with Catmull-Rom, try to find a transformation from cardinal control values to Hermite control values.
This can be used to get the full characteristic matrix.

In [ ]:
control_values_H = sp.Matrix([x0, x1, xd0.name, xd1.name])
control_values_H

From the [notebook about uniform Hermite splines](hermite-uniform.ipynb):

In [ ]:
M_H = NamedMatrix(
    r'{M_\text{H}}',
    sp.S('Matrix([[2, -2, 1, 1], [-3, 3, -2, -1], [0, 0, 1, 0], [1, 0, 0, 0]])'))
M_H

In [ ]:
M_KBtoH = NamedMatrix(r'{M_\text{KB$\to$H}}', 4, 4)
M_KB = NamedMatrix(r'{M_\text{KB}}', M_H.name * M_KBtoH.name)
M_KB

In [ ]:
sp.Eq(control_values_H, M_KBtoH.name * control_values_KB)

If we substitute the above definitions of $\boldsymbol{\dot{x}}_0$ and $\boldsymbol{\dot{x}}_1$, we can directly read off the matrix elements:

In [ ]:
M_KBtoH.expr = sp.Matrix([[expr.coeff(cv) for cv in control_values_KB]
                          for expr in control_values_H.subs([xd0.args, xd1.args]).expand()])
M_KBtoH

In [ ]:
M_KBtoH.pull_out(sp.S.Half)

In [ ]:
M_KB = M_KB.subs([M_H, M_KBtoH]).doit()
M_KB

In [ ]:
M_KB.pull_out(sp.S.Half)

And for completeness' sake, its inverse:

In [ ]:
M_KB.I

In [ ]:
t = sp.symbols('t')

In [ ]:
b_KB = NamedMatrix(r'{b_\text{KB}}', sp.Matrix([t**3, t**2, t, 1]).T * M_KB.expr)
b_KB.T

To be able to plot the basis functions, let's substitute $a$, $b$, $c$ and $d$ back in (which isn't pretty):

In [ ]:
b_KB = b_KB.subs([a, b, c, d]).simplify()
b_KB.T

In [ ]:
sp.plot(*b_KB.expr.subs({T0: 0, T1: 0, C0: 0, C1: 1, B0: 0, B1: 0}), (t, 0, 1));

In [ ]:
sp.plot(*b_KB.expr.subs({T0: 0, T1: 0, C0: 0, C1: -0.5, B0: 0, B1: 0}), (t, 0, 1));

TODO: plot some example curves